# Travel route

This work is to make a travel route app for tourist to make the their journey more easy. It will have an app which showcases a map where the user can select a to and fro destination & get the reviews about the place. The reviews will have details about:

* Road conditions.
* Weather conditions.
* Nearby scenaries like park, garden etc.
* Presence of the hotels & restaurants.
* Traffic conditions.

The procedure is as follows. The user can login in to the app & can mark the to and fro destinations. So the user will be able to view the following things, they are :

* Previous reviews about the route.
* The nearby popular tourist destination.
* Difficulty score associated with the route.

Each user after completing a journey will be asked for a feedback which they can type in the app itself. The user can mention about the above mentioned review details. This will be stored in a database. The database can be a structured db such as **_sql server_** or unstructed no sql dbs like **_mongo db_**. Each review will be saved as a entry in the db. This entry will then be passed into a NLU (Natural Language Understanding) unit which will be used to identify the positive words(less traffic, good scenary, parks..) and negative words(heavy traffic, bad roads..). Depeding on the number of occurence of the positive & negative words, difficulty score will be generated. More negative words means increase in the difficulty score. The difficulty score will be set to vary in between 0-1.



### Dependancies

In [2]:
import os
path = "/home/sooraj/Desktop/Grapes/Travel_route"
os.chdir(path)

Running the below cell once will be enough to install the dependancies. Here the Rasa nlu is installed. This is a machine learning library in python used for the Natural Language Processing (NLP) applications. This library can understand the words in the sentences and extract them & can classify the sentence into different classes as per the user requirment. To detect and understand the english words we also download the english model that was developed using the wikipedia corpus and link this to the library.

In [19]:
!pip install -r /home/sooraj/Desktop/Grapes/Travel_route/requirements.txt
!pip install rasa_nlu
!pip install rasa_nlu[spacy]
!python -m spacy download en_core_web_md
!python -m spacy link en_core_web_md en
!pip install nodejs


    Linking successful
    /home/sooraj/anaconda3/envs/py36/lib/python3.6/site-packages/en_core_web_md
    -->
    /home/sooraj/anaconda3/envs/py36/lib/python3.6/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')




    Link 'en' already exists
    To overwrite an existing link, use the --force flag.

/home/sooraj/.nvm/versions/node/v8.12.0/bin/rasa-nlu-trainer -> /home/sooraj/.nvm/versions/node/v8.12.0/lib/node_modules/rasa-nlu-trainer/server.js
npm WARN slick-carousel@1.8.1 requires a peer of jquery@>=1.8.0 but none is installed. You must install peer dependencies yourself.
npm WARN optional SKIPPING OPTIONAL DEPENDENCY: fsevents@1.2.7 (node_modules/rasa-nlu-trainer/node_modules/fsevents):
npm WARN notsup SKIPPING OPTIONAL DEPENDENCY: Unsupported platform for fsevents@1.2.7: wanted {"os":"darwin","arch":"any"} (current: {"os":"linux","arch":"x64"})

+ rasa-nlu-trainer@0.2.7
added 3 packages from 2 contributors, removed 2 packages and updated 24 packages in 70.846s


### Dataset preparation

Before preparation of the dataset we need to install the _nlu trainer_. This is a web based ui which will help us in the dataset creation. This is supervised learning. So we have to create tagged dataset that will teach the model which sentence belongs to which class and which all words should the model extract. The ui will consists of 2 text boxes. The left one indicates which class the sentence belongs to & the right one indicates the sentence example. Now within this sentence example there is an option to click on individual words and assign them a sub-class. 

For example, the sentence

**_"The roads are very smooth with no traffic"_** fall under the class **review** &  
  **_no traffic_** falls under the sub-class **traffic conditions**
  
 while the sentences like:  
**_"Hey good mornings_** fall under the class **greetings** 

In this manner we can tag the data and extract the entities and intents with which we can calculate the difficulty score.

In [27]:
# Installing the dependancies for nlu trainer (nodejs & node package manager(npm) is needed)
os.chdir(path+"/Misc/rasa-nlu-trainer")
!npm i -g rasa-nlu-trainer

npm ERR! code EBADPLATFORMrefresh-package-json /home/som
npm ERR! notsup Unsupported platform for fsevents@1.2.7: wanted {"os":"darwin","arch":"any"} (current: {"os":"linux","arch":"x64"})
npm ERR! notsup Valid OS:    darwin
npm ERR! notsup Valid Arch:  any
npm ERR! notsup Actual OS:   linux
npm ERR! notsup Actual Arch: x64

npm ERR! A complete log of this run can be found in:
npm ERR!     /home/sooraj/.npm/_logs/2019-02-23T09_58_35_459Z-debug.log
npm WARN optional SKIPPING OPTIONAL DEPENDENCY: fsevents@1.2.4 (node_modules/fsevents):
npm WARN notsup SKIPPING OPTIONAL DEPENDENCY: Unsupported platform for fsevents@1.2.4: wanted {"os":"darwin","arch":"any"} (current: {"os":"linux","arch":"x64"})

+ jquery-1.8@1.8.3
added 1 package from 1 contributor and audited 11864 packages in 26.791s
found 32 vulnerabilities (17 low, 6 moderate, 7 high, 2 critical)
  run `npm audit fix` to fix them, or `npm audit` for details
/home/sooraj/.nvm/versions/node/v8.12.0/bin/rasa-nlu-trainer -> /home/sooraj/

After installation we will launch the nlu trainer using the following command. What this will do is open the **demo-rasa.json** file in ui for easy editing. We can add new key value pairs or edit the existing one in it. After the data tagging & insertion is done, the changes should be saved and the **running process should be stopped using the stop button** in the jupyter notebook. After the current process is stopped, only then we will be able to execute the remaining the cells.

In [3]:
os.chdir(path)
!rasa-nlu-trainer

searching for the training examples...
found /home/sooraj/Desktop/Grapes/Travel_route/projects/default/model_20190223-145829/training_data.json
server listening at http://localhost:41141/
^C


Loading the packages and then displaying the training data which is **demo-rasa.json**

In [1]:
# Load the Packages
from rasa_nlu.training_data  import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [4]:
!cat demo-rasa.json

{
  "rasa_nlu_data": {
    "regex_features": [
      {
        "name": "zipcode",
        "pattern": "[0-9]{5}"
      },
      {
        "name": "greet",
        "pattern": "hey[^\\s]*"
      }
    ],
    "entity_synonyms": [
      {
        "value": "chinese",
        "synonyms": ["Chinese", "Chines", "chines"]
      },
      {
        "value": "vegetarian",
        "synonyms": ["veggie", "vegg"]
      }
    ],
    "common_examples": [
      {
        "text": "hey",
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "howdy",
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hey there",
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hello", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hi", 
        "intent": "greet", 
        "entities": []
      },
      {
        "text": "good morning",
        "intent": "greet",
        "entities": []

Loading the json data set and then using the spacy machine learning library as backed. Spacy is the main component userd here for feature extraction. The finally the json data is trained. 

In [6]:
# Loading DataSet
train_data = load_data('demo-rasa.json')

In [8]:
# Config Backend using Sklearn and Spacy
trainer = Trainer(config.load("config_spacy.yml"))

In [9]:
# Training Data
trainer.train(train_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


The trained model is saved in the projects directory. The model will be loaded in the coming steps. This trained model will be then used for testing purposes.  

In [11]:
# Returns the directory the model is stored in (Creat a folder to store model in)
model_directory = trainer.persist('/home/sooraj/Desktop/Grapes/Travel_route/projects')

In [12]:
import spacy
nlp = spacy.load('en')

In [15]:
from rasa_nlu.model import Metadata, Interpreter

In [16]:
# where `model_directory points to the folder the model is persisted in
interpreter = Interpreter.load(model_directory)

In [17]:
# Prediction of Intent
interpreter.parse(u"I am looking for an Italian Restaurant where I can eat")

{'intent': {'name': 'restaurant_search', 'confidence': 0.766792140872358},
 'entities': [],
 'intent_ranking': [{'name': 'restaurant_search',
   'confidence': 0.766792140872358},
  {'name': 'goodbye', 'confidence': 0.11751034779156634},
  {'name': 'affirm', 'confidence': 0.061814362053137464},
  {'name': 'greet', 'confidence': 0.053883149282938074}],
 'text': 'I am looking for an Italian Restaurant where I can eat'}